<a href="https://colab.research.google.com/github/Nenkoff/examples/blob/main/SentimentAnalysis_TRIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00


In [2]:
tokenizer = AutoTokenizer.from_pretrained('Geotrend/bert-base-bg-cased')

In [11]:
# prompt = 'Войната по пътищата е безмилостна! Двама души загинаха днес на автомагистрала Тракия.'
# prompt = 'Успех за България! Наши ученици спечелиха престижни награди на европейски конкурс за млади учени.'
# prompt = '<p>На тротоара, отдясно по посоката на движение на бул. "Прага" от Петте Кьошета към бул. "Пенчо Славейков" има засадени фиданки. На маркираното място на две отделни фиданки подпорите, които държат фиданката да расте права, са силно наклонени. Така те противоестествено ги карат да растат в грешна посока, макар че стеблата да се опитват да се изправят. Така се получават много вредни за дървото и неестетични изкривявания по стеблата.'
prompt = '<p>На тротоара, отдясно по посоката на движение на бул. &quot;Панчо Владигеров&quot; от бул. &quot;Сливница&quot; към&nbsp;Обеля има засадени фиданки. На&nbsp;няколко (поне четири)&nbsp;от фиданките на маркираното място, подпорите, които държат фиданката да расте права, са видимо наклонени.&nbsp;Явно е, че е необходимо да има периодична проверка на подпорите. Те противоестествено карат фиданките да растат в грешна посока, макар че стеблата се опитват да се изправят. Така се получават много вредни за дървото и неестетични изкривявания по стеблата. И веднъж изкривено, стеблото остава така за целия живот на дървото.</p>'

In [13]:
inputs = tokenizer.encode_plus(prompt, None, add_special_tokens=True, max_length=200, pad_to_max_length=True, return_token_type_ids=True)
print(inputs)

{'input_ids': [11, 41, 92, 43, 675, 213, 6146, 8324, 827, 25, 431, 6034, 3900, 406, 406, 6518, 2557, 256, 5667, 256, 7167, 27, 19, 6924, 257, 40, 6732, 382, 5237, 167, 8671, 4460, 500, 19, 6924, 257, 40, 431, 7167, 27, 19, 6924, 257, 40, 182, 7570, 2632, 19, 6924, 257, 40, 2353, 19, 90, 3049, 487, 40, 2918, 4081, 1591, 354, 9440, 2153, 215, 6377, 4021, 27, 675, 19, 90, 3049, 487, 40, 5027, 21, 406, 631, 4622, 22, 19, 90, 3049, 487, 40, 431, 215, 6377, 4021, 632, 256, 8570, 8236, 661, 5122, 25, 995, 6192, 1487, 25, 2365, 199, 9049, 446, 215, 6377, 3511, 460, 483, 211, 4771, 332, 2625, 25, 721, 6879, 3110, 256, 8739, 11547, 27, 19, 90, 3049, 487, 40, 194, 3784, 200, 25, 2100, 200, 7914, 483, 1591, 2357, 6193, 1501, 4945, 551, 256, 995, 6192, 1487, 27, 5924, 1740, 2594, 4884, 7831, 8151, 446, 215, 6377, 4021, 632, 483, 211, 4771, 1399, 197, 7672, 4870, 486, 406, 6518, 310, 25, 11221, 6005, 2100, 2332, 8023, 3821, 391, 209, 9876, 2901, 483, 391, 472, 9578, 9560, 27, 5309, 391, 5699, 446, 2

In [5]:
import torch

In [6]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('Geotrend/bert-base-bg-cased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)

    def forward(self, ids, mask, token_type_ids):
        output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids).pooler_output
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

In [7]:
!gdown "1RamIB822hKPQ98xsAQLb1hOctqFvniqa&confirm=t"
model = torch.load('sentiment-bg-model', map_location=torch.device('cpu'))

Downloading...
From: https://drive.google.com/uc?id=1RamIB822hKPQ98xsAQLb1hOctqFvniqa&confirm=t
To: /content/sentiment-bg-model
100% 382M/382M [00:10<00:00, 36.0MB/s]


In [8]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
torch.cuda.empty_cache()


In [14]:
ids = torch.tensor(inputs['input_ids'], dtype=torch.long).unsqueeze(0).to(device)
mask = torch.tensor(inputs['attention_mask'], dtype=torch.long).unsqueeze(0).to(device)
token_type_ids = torch.tensor(inputs["token_type_ids"], dtype=torch.long).unsqueeze(0).to(device)

In [15]:
output = model(ids, mask, token_type_ids)
['NEGATIVE', 'NEUTRAL', 'POSITIVE'][torch.argmax(output)]

'NEUTRAL'